# Dealing with variable-length sequences efficiently

This notebook is meant to be run using a GPU.

RNNs can work with variable length sequences. This notebook shows how to do this efficiently with `keras`.

In [ ]:
import os
from os.path import exists
from urllib.request import urlretrieve

import tensorflow as tf
import numpy as np
import pandas
from tqdm.auto import tqdm

For this hands-on we will use the [Top Tagging dataset](https://arxiv.org/abs/1707.08966). This simplified MC data consists of signal Top jets and other quark and gluon background jets. For each jet, a maximum of 200 constituents were identified and ranked according their $P_T$. The four-momenta of these ranked constituents are now used as an input sequence for our RNN. Note that the number of constituents varies between the jets.

At first we will download the data and have a look at it.

In [ ]:
if not exists("test.h5"):
    urlretrieve("https://desycloud.desy.de/index.php/s/llbX3zpLhazgPJ6/download?path=%2F&files=test.h5", "test.h5")

In [ ]:
store = pandas.HDFStore("test.h5")
df = store.select("table", stop=100000) # Read the first X events

truth = df.iloc[:, -6:]
df = df.iloc[: , :-6] # drop the last six columns with truth information
df.head()

In [ ]:
truth.head()

In order to have a dataset as if we measured it, we will remove all padded zeros from the dataset..

In [ ]:
data = df.to_numpy()
data = data.reshape(len(df), -1, 4) # last dimension will be (E, px, py, pz)
data = [row[~(row == 0).all(axis=-1)] for row in data] # remove constituents with all momentum components == 0

In [ ]:
len(data[0])

Our dataset is now a list of measured jets consisting of arrays with shape (constituent, feature). Or more general: a list (batch) of arrays (sequence, feature). The number of constituents varies between the jets.

In [ ]:
for jet in range(5):
    print(data[jet].shape[0], 'constituents for jet nr.', jet)

We want to feed this through the following 2-layer LSTM model - it takes a batch of sequences of arbitrary length and outputs a batch of numbers:

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(None, 4), return_sequences=True),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(1),
])

In [ ]:
model.summary()

How do we feed in the variable number of constituents? Well, since the first two input dimensions of our model are unspecified (jets, constituent) we can pass each jet separately. Let's see how fast this is.

In [ ]:
for jet in tqdm(data):
    model(jet[np.newaxis, :])

Doesn't seem that bad does it?

Wait! We haven't seen yet how fast it could be ...

If you look at the GPU utilization (e.g. with `nvidia-smi`) while this is running you will see it is rather low. That's because RNNs are inherently sequential - we can't process the different steps of a sequence (of constituents in jet) in parallel.

But what we can do is to process each step of the sequence (each constituent in jet) in parallel across all jets in our batch!

Keras will do this if we provide batches that are Tensors of fixed length.

To try this out, let's enlarge the sequences to a fixed length and fill missing values with zeros:

In [ ]:
padded_data = tf.keras.preprocessing.sequence.pad_sequences(data, padding="post", dtype="float32")

Now we have a dataset with a uniform number of constituents (again).

In [ ]:
padded_data.shape # (batch, constituents, features)

In [ ]:
padded_data[0,:3,:] # four-momenta of the first three constituents in the first jet

Now evaluate the RNN, but this time parallelize the evaluation of the sequence (constituents) over the batch (jet) dimension.

In [ ]:
model.predict(padded_data, batch_size=256, verbose=True)[:10,:]

That should have been **much** faster.

But now the model also processed the 0-padded values. We can see that e.g. the first output is different than what we expect from passing in the first sequence:

In [ ]:
model(data[0][np.newaxis, :])

In keras we can solve this by a `Masking` layer - subsequent RNN layers will respect this and only process non-masked inputs.

For more info, see https://keras.io/guides/understanding_masking_and_padding/

In [ ]:
masked_model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=0.0),
    tf.keras.layers.LSTM(128, input_shape=(None, 4), return_sequences=True),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(1),
])

In [ ]:
masked_model.build(input_shape=(None, None, 4))

In [ ]:
# set the weights such that we can compare the outputs of both models
masked_model.set_weights(model.get_weights())

In [ ]:
masked_model.predict(padded_data, batch_size=256, verbose=True)[:10,:]

This time the output is compatible with the one-by-one processing.

# Exercise: Try to distinguish top quark from QCD jets using an RNN

The dataset comes with a label, indicating whether a jet comes from a top quark decay or from gluons (QCD):

In [ ]:
y = truth.is_signal_new.to_numpy()
X = padded_data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

Try to train an RNN-based model to predict the correct label:

In [ ]:
model = ...
model.compile(...)
model.summary()

In [ ]:
history = model.fit(...)

In [ ]:
pandas.DataFrame(history.history).plot()
plt.xlabel("epoch")

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
y_pred = model.predict(X_test, batch_size=256)

In [ ]:
fpr, tpr, thr = roc_curve(y_test, y_pred)

In [ ]:
plt.plot(tpr, 1 / fpr)
plt.yscale("log")
plt.ylabel("QCD jet rejection")
plt.xlabel("Top quark jet efficiency")
plt.yscale("log")